In [ ]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()

In [ ]:
query1 = """
--Shows all patients of a doctor
select
s.name as Zugewiesene_Station,
p.name as Patient_Name,
p.geburtsdatum as Geburtsdatum_Patient,
a.name as Arzt_Name,
b.beschreibung as Behandlung_Beschreibung,
m.wirkstoff as Wirkstoff

FROM arzt a
JOIN arzt_behandlung ab ON a.arzt_id = ab.arzt_id
JOIN behandlung b ON ab.behandlungs_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
JOIN behandlung_medikament bm on b.behandlungs_id = bm.behandlungs_id
JOIN medikament m on m.medikament_id = bm.medikament_id
JOIN Station s on s.station_id = a.station_id


WHERE a.arzt_id = 2 and b.abgeschlossen = false;
"""

In [ ]:
query2 = """
-- Shows all patients of a nurse (Pflegekraft)

SELECT
    s.name AS Zugewiesene_Station,
    pk.name AS Pflegekraft_Name,
    p.name AS Patient_Name,
    p.geburtsdatum AS Geburtsdatum_Patient,
    b.beschreibung AS Behandlung_Beschreibung,
    m.wirkstoff AS Wirkstoff

FROM pflegekraft pk
JOIN assistenz a ON pk.pflege_id = a.pflegekraft_id
JOIN assistenz_behandlung ab ON a.assistenz_id = ab.assistenz_id
JOIN behandlung b ON ab.behandlungs_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
JOIN behandlung_medikament bm ON b.behandlungs_id = bm.behandlungs_id
JOIN medikament m ON m.medikament_id = bm.medikament_id
JOIN station s ON s.station_id = pk.station_id

WHERE pk.pflege_id = 3   -- Beispiel: Pflegekraft 3
  AND b.abgeschlossen = FALSE;
  """

In [ ]:
query3 = """
-- shows which station uses which drug how often
SELECT
    s.name as Station,
    m.wirkstoff as Medikament,
    COUNT(*) AS verwendungen
FROM behandlung_medikament bm
JOIN medikament m ON bm.medikament_id = m.medikament_id
JOIN behandlung b ON bm.behandlungs_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
JOIN station s ON p.station_id = s.station_id
GROUP BY s.name, m.wirkstoff
ORDER BY s.name, verwendungen DESC;
"""

In [ ]:
query4 = """
-- zeigt wie viele behandlungen ein arzt hat

SELECT 
    a.Arzt_ID,
    a.Name AS Arztname,
    COUNT(ab.Behandlungs_ID) AS Anzahl_Behandlungen
FROM Arzt a
LEFT JOIN Arzt_Behandlung ab 
    ON a.Arzt_ID = ab.Arzt_ID
GROUP BY a.Arzt_ID, a.Name
ORDER BY COUNT(ab.Behandlungs_ID) DESC;
"""

In [ ]:
query5 = """
-- anzahl belegte betten
SELECT 
    s.Station_ID,
    s.Name AS Station,
    COUNT(*) AS Gesamtbetten,
    COUNT(CASE WHEN b.Status = 'belegt' THEN 1 END) AS Belegte_Betten,
    COUNT(CASE WHEN b.Status = 'frei' THEN 1 END) AS Freie_Betten
FROM Station s
JOIN Bett b ON b.Station_ID = s.Station_ID
GROUP BY s.Station_ID, s.Name
ORDER BY s.Station_ID;
"""